In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf


In [3]:
data = pd.read_csv('/content/drive/MyDrive/NLP/df_and_eda.csv')

In [6]:
df = data

### Поиск по описанию ver.2 ( DistilBert)


In [ ]:
# Скачиваем всё что надо и не надо
!pip install sentence-transformers

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
#обучаем модели DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
data_list = df['description'].tolist()

In [8]:
#Получаем эмбеддинги для наших описаний фильмов

data_embeddings = model.encode(data_list, show_progress_bar=True)

Batches:   0%|          | 0/1695 [00:00<?, ?it/s]

In [34]:
#Наш запрос
query = "Zombies eating human brains and destroy the city"

query_embedding = model.encode([query])[0]

# косинусное сходство между запросом и всеми описаниями фильмов
similarities = cosine_similarity([query_embedding], data_embeddings)

# Индексы наиболее похожих описаний
top_indices = np.argsort(similarities[0])[::-1][:5].tolist()

#результаты
results = []
for index in top_indices:
    film_title = df.iloc[index]['title']
    similarity_score = similarities[0][index]
    results.append((index, film_title, similarity_score))


for result in results:
    index, _, similarity = result
    print(index, _, similarity)


9102 Zombie Bloodbath 3: Zombie Armageddon 0.8203016
28511 Dead Country 0.7496003
45242 Call of Duty: Advanced Warfare - Exo Zombies 0.73869777
28716 Come Get Some! 0.7359661
27506 East Side Zombies 0.7267505


Итак, получая наши эмбеддинги через модель distilBERT мы добились косинусного сходства чуть повыше на том же запрос, если сравнивать с Word2Vec.

Но что действительно важно, так это то, что distilBERT учитывает семантику, контекст и смысл, что выводит нам фильмы всё-таки похожие на наш запрос.
